In [2]:
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:

model = torch.load( "model.pth")
for ele in model:#turn to numpy
    model[ele]=model[ele].cpu().numpy()
#turn float to int
for layers in model:
    try:
        model[layers].shape[1]## if parameter is 2D
        row,col=model[layers].shape
        for row in range(model[layers].shape[0]):
            for col in range(model[layers].shape[1]):
                model[layers][row][col]=int((model[layers][row][col]+1)/2*255-127)
    except:
        #model[layers].shape #1D
        # print(model[layers].shape[0])
        for i in range(model[layers].shape[0]):
            model[layers][i]=int((model[layers][i]+1)/2*255-127)


In [4]:
model_declared_name_list=["in_buf_weight0","weight0_bias0","bias0_relu0",
                            "relu0_weight2","weight2_bias2","bias2_relu2",
                            "relu2_weight4","weight4_bias4"]
model_layer_name_list=[]
model_size_list=[]
img_size=784

for layers in model:
    model_layer_name_list.append(layers)
    model_size_list.append(model[layers].shape)

print(model_declared_name_list)
print(model_size_list)
print(model_layer_name_list)



['in_buf_weight0', 'weight0_bias0', 'bias0_relu0', 'relu0_weight2', 'weight2_bias2', 'bias2_relu2', 'relu2_weight4', 'weight4_bias4']
[(64, 784), (64,), (32, 64), (32,), (10, 32), (10,)]
['linear_relu_stack.0.weight', 'linear_relu_stack.0.bias', 'linear_relu_stack.2.weight', 'linear_relu_stack.2.bias', 'linear_relu_stack.4.weight', 'linear_relu_stack.4.bias']


In [5]:
f=open("verilog/wire_connection.v","w")
#input buffer to weight0
parameter_idx=0
model_name_idx=0
wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
current_layer_parameter=model[model_layer_name_list[parameter_idx]]
f.write("\n")

#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
    
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=")
    for in_buf_idx in range(img_size):
        if(in_buf_idx==img_size-1):
            f.write("in_buf["+str(in_buf_idx)+"]*("+str(int(current_layer_parameter[naming_idx][in_buf_idx]))+")")
        else:
            f.write("in_buf["+str(in_buf_idx)+"]*("+str(int(current_layer_parameter[naming_idx][in_buf_idx]))+")+")

    f.write(";\n")
#weight0 to bias0
parameter_idx+=1
model_name_idx+=1
wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
current_layer_parameter=model[model_layer_name_list[parameter_idx]]
previous_layer_name=model_declared_name_list[model_name_idx-1]
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=")
    f.write(previous_layer_name+str(naming_idx))
    f.write("+(")
    f.write(str(int(current_layer_parameter[naming_idx])))

    f.write(");\n")
#bias0 to relu0
model_name_idx+=1
#parameter_idx does not change, Relu does not have parameter
wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
previous_layer_name=model_declared_name_list[model_name_idx-1]
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=(")
    f.write(previous_layer_name+str(naming_idx))
    f.write("[DATAWIDTH-1]==1'b1)   ?   ")
    f.write("{DATAWIDTH{1'b0}}:")
    f.write(previous_layer_name+str(naming_idx))
    f.write(";\n")

#relu0 to weight2
parameter_idx+=1
model_name_idx+=1

wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
current_layer_parameter=model[model_layer_name_list[parameter_idx]]
previous_layer_name=model_declared_name_list[model_name_idx-1]
previous_layer_sz=model_size_list[parameter_idx-1][0]
f.write("\n")
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=")
    for previoud_layer_idx in range(previous_layer_sz):
        if(previoud_layer_idx==previous_layer_sz-1):
            f.write(previous_layer_name+str(previoud_layer_idx))
            f.write("*("+str(int(current_layer_parameter[naming_idx][previoud_layer_idx]))+")")
        else:
            f.write(previous_layer_name+str(previoud_layer_idx))
            f.write("*("+str(int(current_layer_parameter[naming_idx][previoud_layer_idx]))+")+")
    f.write(";\n")
#weight2 to bias2
parameter_idx+=1
model_name_idx+=1
wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
current_layer_parameter=model[model_layer_name_list[parameter_idx]]
previous_layer_name=model_declared_name_list[model_name_idx-1]
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=")
    f.write(previous_layer_name+str(naming_idx))
    f.write("+(")
    f.write(str(int(current_layer_parameter[naming_idx])))

    f.write(");\n")
#bias2 to relu2
model_name_idx+=1
#parameter_idx does not change, Relu does not have parameter
wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
previous_layer_name=model_declared_name_list[model_name_idx-1]
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=(")
    f.write(previous_layer_name+str(naming_idx))
    f.write("[DATAWIDTH-1]==1'b1)   ?   ")
    f.write("{DATAWIDTH{1'b0}}:")
    f.write(previous_layer_name+str(naming_idx))
    f.write(";\n")
#relu2 to weight4
parameter_idx+=1
model_name_idx+=1

wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
current_layer_parameter=model[model_layer_name_list[parameter_idx]]
previous_layer_name=model_declared_name_list[model_name_idx-1]
previous_layer_sz=model_size_list[parameter_idx-1][0]
f.write("\n")
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=")
    for previoud_layer_idx in range(previous_layer_sz):
        if(previoud_layer_idx==previous_layer_sz-1):
            f.write(previous_layer_name+str(previoud_layer_idx))
            f.write("*("+str(int(current_layer_parameter[naming_idx][previoud_layer_idx]))+")")
        else:
            f.write(previous_layer_name+str(previoud_layer_idx))
            f.write("*("+str(int(current_layer_parameter[naming_idx][previoud_layer_idx]))+")+")
    f.write(";\n")
# weight4 to bias4 
parameter_idx+=1
model_name_idx+=1
wire_cnt=model_size_list[parameter_idx][0]
current_model_name=model_declared_name_list[model_name_idx]
current_layer_parameter=model[model_layer_name_list[parameter_idx]]
previous_layer_name=model_declared_name_list[model_name_idx-1]
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATAWIDTH-1:0]   ")   
    f.write(current_model_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+current_model_name+str(naming_idx)+"=")
    f.write(previous_layer_name+str(naming_idx))
    f.write("+(")
    f.write(str(int(current_layer_parameter[naming_idx])))

    f.write(");\n")
f.close()

In [6]:
#in_buf assignment
img_size=784
f=open("verilog/in_buff.v","w")

for i in range(img_size):
    if(i==img_size-1):
        f.write("in_buf["+str(i)+"]<=img"+str(i)+";\n")
    else:
        f.write("in_buf["+str(i)+"]<=img"+str(i)+";")
f.close()

In [7]:
data_width=64
buffer_cnt=2
input_data_width=8
output_node=10
img_size=784

#get_bin = lambda x: format(x, 'b').zfill(fsm_bit)

f=open("verilog/TCB_net.v","w")
f.write("`timescale 1ns / 1ps\n")
f.write("module net\n")
f.write("#(\n")
f.write("DATAWIDTH="+str(data_width)+",\n")
f.write("BUFFCNT="+str(buffer_cnt)+",\n")
f.write("IMG_SZ="+str(img_size)+"\n")
f.write(")\n")
f.write("(\n")
f.write("input clk,\n")
f.write("input rst,\n")
##input
f.write("input  ["+str(input_data_width-1)+":0] ")
for i in range(img_size):
    if(i==img_size-1):
        f.write("img"+str(i)+",\n")
    else:
        f.write("img"+str(i)+",")
f.write("\n")
##output 
f.write("output    reg [4-1:0]predict\n")

f.write("\n")
f.write(");\n")
#port declaration end
f.write("reg    [DATAWIDTH-1:0] in_buf  [0:"+str(img_size-1)+"];\n")

f.write("reg    [DATAWIDTH-1:0] result[0:"+str(output_node-1)+"];\n")

f.write("/////Start Declatation////////\n")

f.write("\n")
f.write("/////Eed Declatation////////\n")

f.write("integer i;\n")
f.write("always@(posedge clk or posedge rst)\n")
f.write("   begin\n")
f.write("       if(rst)\n")
f.write("           begin\n")
f.write("           for(i=0;i<"+str(img_size)+";i=i+1)\n")
f.write("               begin\n")
f.write("                   in_buf[i]<=0;\n")
f.write("               end\n")
f.write("           for(i=0;i<"+str(output_node)+";i=i+1)\n")
f.write("               begin\n")
f.write("                   result[i]<=0;\n")
f.write("               end\n")
f.write("           end\n")
f.write("       else\n")
f.write("       begin\n           ")

f.write("`include ")
f.write("\"")
f.write("in_buff.v")
f.write("\"")
f.write("\n")

#output reg assignment 
last_layer_name =model_declared_name_list[-1]
for out_idx in range(output_node):
    f.write("            result["+str(out_idx)+"]<="+last_layer_name+str(out_idx)+";\n")

f.write("           predict <=com_re01234567_89[4+DATAWIDTH-1:4+DATAWIDTH-1-3];")
f.write("       end\n")
f.write("end\n")
f.write("\n")
f.write("\n")
f.write("//wire declaration \n")

f.write("\n")
f.write("endmodule")
f.close()

In [8]:
//module
wire [4+DATAWIDTH-1:0] com_re01,com_re23,com_re45,com_re67,com_re89;
//check positive or negtive
assign com_re01=(result[0][DATAWIDTH-1]^result[1][DATAWIDTH-1]) ? 
                                                        ((result[0][DATAWIDTH-1]==1'b0)   ?   {4'd0,result[0]}:{4'd1,result[1]}):
                                                        ((result[0]>result[1]) ? {4'd0,result[0]}:{4'd1,result[1]});
assign com_re23=(result[2][DATAWIDTH-1]^result[3][DATAWIDTH-1]) ? 
                                                        ((result[2][DATAWIDTH-1]==1'b0)   ?   {4'd2,result[2]}:{4'd3,result[3]}):
                                                        ((result[2]>result[3]) ? {4'd2,result[2]}:{4'd3,result[3]});
assign com_re45=(result[4][DATAWIDTH-1]^result[5][DATAWIDTH-1]) ? 
                                                        ((result[4][DATAWIDTH-1]==1'b0)   ?   {4'd4,result[4]}:{4'd5,result[5]}):
                                                        ((result[4]>result[5]) ? {4'd4,result[4]}:{4'd5,result[5]});                                                        
assign com_re67=(result[6][DATAWIDTH-1]^result[7][DATAWIDTH-1]) ? 
                                                        ((result[6][DATAWIDTH-1]==1'b0)   ?   {4'd6,result[6]}:{4'd7,result[7]}):
                                                        ((result[6]>result[7]) ? {4'd6,result[6]}:{4'd7,result[7]});
assign com_re89=(result[8][DATAWIDTH-1]^result[9][DATAWIDTH-1]) ? 
                                                        ((result[8][DATAWIDTH-1]==1'b0)   ?   {4'd8,result[8]}:{4'd9,result[9]}):
                                                        ((result[8]>result[9]) ? {4'd8,result[8]}:{4'd9,result[9]});

wire [4+DATAWIDTH-1:0] com_re01_23,com_re45_67,com_re0123_4567,com_re01234567_89;
assign com_re01_23=(com_re01[DATAWIDTH-1]^com_re23[DATAWIDTH-1])   ?
                                                        ((com_re01[DATAWIDTH-1]==1'b0)  ?   com_re01:com_re23):
                                                        ((com_re01[DATAWIDTH-1:0]>com_re23[DATAWIDTH-1:0]) ?   com_re01:com_re23);
assign com_re45_67=(com_re45[DATAWIDTH-1]^com_re67[DATAWIDTH-1])   ?
                                                        ((com_re45[DATAWIDTH-1]==1'b0)  ?   com_re45:com_re67):
                                                        ((com_re45[DATAWIDTH-1:0]>com_re67[DATAWIDTH-1:0]) ?   com_re45:com_re67);
assign com_re0123_4567=(com_re01_23[DATAWIDTH-1]^com_re45_67[DATAWIDTH-1])   ?
                                                        ((com_re01_23[DATAWIDTH-1]==1'b0)  ?   com_re01_23:com_re45_67):
                                                        ((com_re01_23[DATAWIDTH-1:0]>com_re45_67[DATAWIDTH-1:0]) ?   com_re01_23:com_re45_67);
assign com_re01234567_89=(com_re0123_4567[DATAWIDTH-1]^com_re89[DATAWIDTH-1])   ?
                                                        ((com_re0123_4567[DATAWIDTH-1]==1'b0)  ?   com_re0123_4567:com_re89):
                                                        ((com_re0123_4567[DATAWIDTH-1:0]>com_re89[DATAWIDTH-1:0]) ?   com_re0123_4567:com_re89);



//assign predict =com_re01234567_89[4+DATAWIDTH-1:4+DATAWIDTH-1-3];

SyntaxError: invalid syntax (1822985077.py, line 1)

In [ ]:
s=

In [9]:
#turn img from float to int
def int_conver(np_a):
    for idx,ele in enumerate(np_a):
        np_a[idx]=int((ele+1)/2*255-127)

    return np_a

In [ ]:
test_size=100
f=open("verilog/tb.v","w")
f.write("`timescale 1ns / 1ps\n")

f.write("module tb;\n")
f.write("parameter DATAWIDTH =64;\n")
f.write("reg rst,clk;\n")
for i in range(img_size):
    f.write("reg [7:0]  img"+str(i)+";")
f.write("\n")
f.write("wire [3:0] predict;\n")

f.write("       net #(64,2,784) \n")
f.write("       UDT\n")
f.write("      (.clk(clk),\n")
f.write("       .rst(rst),\n        ")

for i in range(img_size):
    f.write(".img"+str(i)+"(img"+str(i)+"),")
f.write("\n")
f.write("       .predict(predict)\n")
f.write("       );\n")
#port end



f.write("always #5 clk=~clk;\n")

#read label for test acc
f.write("reg[3:0] read_data [0:"+str(test_size)+"];\n")
f.write("integer i,hit;\n")
f.write("\n")
f.write("initial \n")
f.write("   begin\n")
f.write("       i=-4;hit=0;\n")
f.write("       $display(\"begin read label file... \");\n")
f.write("       $readmemh(\"label.mem\",read_data);\n")
f.write("   end\n")

f.write("always@(posedge clk)\n")
f.write("   begin\n")
f.write("       $display(\"data     :%d predict    :%d\",read_data[i],predict); \n")
f.write("           if(predict==read_data[i])\n")
f.write("               begin\n")
f.write("                   hit=hit+1;\n")
f.write("               end\n")
f.write("       i=i+1'b1;\n")
f.write("   end\n")
f.write("initial \n")
f.write("begin\n")
f.write("rst=1'b1;clk=1'b0;\n")
# for i in range(img_size):
#     f.write("img"+str(i)+"=1'b1;")
f.write("\n")
f.write("#10 rst=1'b0;\n")

for i in range(test_size):
    img,label=test_data[i]
    img=img.reshape(-1)
    img=int_conver(img)
    img=img.cpu().numpy()
    idx=0
    f.write("@(negedge clk)#(10/4) ")
    for ele in img:
        f.write("img")
        f.write(str(idx))
        f.write("=8'd")
        f.write(str(int(ele)))
        f.write(";")
        idx+=1
    f.write("\n")

f.write("$display(\"hit: %d :\",hit);\n")
f.write("#100 $finish;\n")
f.write("end\n")
f.write("\n")

f.write("endmodule \n")
f.close()

In [ ]:
f=open("verilog/label.mem","w")
for i in range(test_size):
    img,label=test_data[i]
    f.write(str(label))
    f.write("\n")
f.close()